In [1]:
# 加载 .env 到环境变量
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

# langchain主要生态

- langchain 模块：以LCEL方式开发DAG结构的chain，也可以作为访问远程chain的python客户端
- langgraph 模块：让LCEL支持循环结构的chain
- langserve 模块：将chain转化为API
- langchain-js 模块：提供langchain的`js`版本，也可以作为访问远程chain的JS客户端
- langsmith 模块：跟踪和调试的平台
- langfuse 模块：langsmith的开源平替

# langchain模块的代码结构

- langchain：入口主模块
- core：核心模块
- community：社区贡献
- partners：合作伙伴模块
- experimetal：实验性模块
- cli：命令行

# 快速开始

In [ ]:
# 请使用最新的open包
!pip install langchain-openai

## 定义大模型

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI().bind(seed=42)

In [41]:
text = "请帮我想一想，生产彩色铅笔的公司有什么好名字?"
llm.invoke(text)

AIMessage(content='1. 彩虹铅笔有限公司\n2. 色彩创意铅笔厂\n3. 彩绘铅笔制造厂\n4. 彩色梦想铅笔公司\n5. 色彩世界铅笔厂\n6. 绚丽铅笔制造有限公司\n7. 色彩艺术铅笔厂\n8. 彩虹色铅笔创意工场\n9. 色彩创意笔具有限公司\n10. 绚烂铅笔艺术厂')

## 定义提示语

In [44]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("请帮我想一想，生产{product}的公司有什么好名字?")
prompt.format(product="彩色铅笔")

'请帮我想一想，生产彩色铅笔的公司有什么好名字?'

In [7]:
from langchain.prompts.chat import ChatPromptTemplate

template = "你是一个有用的翻译助手，从{input_language}语言翻译到{output_language}语言."
human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template),
])

chat_prompt.format_messages(input_language="英文", output_language="中文", text="I love programming.")

[SystemMessage(content='你是一个有用的翻译助手，从英文语言翻译到中文语言.'),
 HumanMessage(content='I love programming.')]

## 使用LCEL

查看不加解析器的生成结果：

In [72]:
resp = (prompt | llm).invoke({"product": "颜色"})
resp

AIMessage(content='红色, 蓝色, 黄色, 绿色, 紫色')

In [74]:
# 提取文本
resp.content

'红色, 蓝色, 黄色, 绿色, 紫色'

你也可以使用解析器直接提取文本：

In [75]:
# 注意 StrOutputParser 属于基础解析器，所以与 CommaSeparatedListOutputParser 位置不同
from langchain.schema.output_parser import StrOutputParser
(prompt | llm | StrOutputParser()).invoke({"product": "颜色"})

'红色, 蓝色, 黄色, 绿色, 紫色'

## 按格式提取文本

In [77]:
from langchain.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()

In [80]:
# 这个解析器的基本能力就是按逗号分解字符串，生成一个列表
output_parser.parse("hi, bye")

['hi', 'bye']

使用这个输出解析器时，通常你要告诉大模型按照期待的逗号间隔的方式生成文本。<br>
因此，你可以使用`get_format_instructions`生成的格式提示信息插入到提示语中。

In [65]:
output_parser.get_format_instructions()

'Your response should be a list of comma separated values, eg: `foo, bar, baz`'

In [82]:
# 组装提示模板
template = ChatPromptTemplate.from_template("生成5个关于{product}的列表.\n\n{format_instructions}")
prompt = template.partial(format_instructions=output_parser.get_format_instructions())

chain = prompt | llm | output_parser
chain.invoke({"product": "颜色"})

['红色', '蓝色', '黄色', '绿色', '紫色']

# 主要概念

## 模型

请记住：OpenAI模型对提示语中包含JSON的情况非常友好。

## 消息

- HumanMessage： 一般是纯文字内容
- AIMessage： 可能包含additional_kwargs，例如 funciton calling 提示
- SystemMessage：部份模型支持的内容提示
- FunctionMessage：函数调用的名称和参数
- ToolMessage：工具调用结果（与FunctionMessage不同）

## 提示语

- PromptValue
- PromptTemplate
- MessagePromptTemplate
- MessagesPlaceholder
- ChatPromptTemplate

## Output Parsers

- StrOutputParser：仅输出字符串；如果输出是 ChatModel，它会仅输出Message的content属性
- OpenAI Functions Parsers：处理OpenAI函数调用所需的函数名和参数
- Agent Output Parsers：帮助智能体解析执行计划

In [8]:
from langchain.schema.output_parser import StrOutputParser
parser = StrOutputParser()
response = chat_prompt.format_messages(input_language="English", output_language="French", text="I love programming.")
print(response)
print(parser.parse(response))

[SystemMessage(content='你是一个有用的翻译助手，从English语言翻译到French语言.'), HumanMessage(content='I love programming.')]
[SystemMessage(content='你是一个有用的翻译助手，从English语言翻译到French语言.'), HumanMessage(content='I love programming.')]


## Prompt封装

In [9]:
# 简单的例子
from langchain.prompts import PromptTemplate

template = PromptTemplate.from_template("给我讲个关于{subject}的笑话")
print(template)
print(template.format(subject='小明'))

input_variables=['subject'] template='给我讲个关于{subject}的笑话'
给我讲个关于小明的笑话


### 字符串模板提示语

#### 基本语法：PromptTemplate.from_template

使用字符串提示时，每个模板都会连接在一起。<br>
您可以直接使用prompt模板或字符串（但列表中的第一个元素必须是prompt模板类型）。

在 langchain 内部，Prompt模板使用 jinja 模板的语法，主要是使用 "{}" 元素。

In [4]:
# 提示语模板与字符串可以直接相加，简化模板构造
from langchain.prompts import PromptTemplate
prompt = (
    PromptTemplate.from_template("告诉我一个关于{topic}的笑话")
    + ", 一定要特别好笑"
    + "\n\使用 {language} 表述"
)
prompt

PromptTemplate(input_variables=['language', 'topic'], template='告诉我一个关于{topic}的笑话, 一定要特别好笑\n\\使用 {language} 表述')

In [20]:
prompt.format(topic="足球", language="中文")

'告诉我一个关于足球的笑话, 一定要特别好笑\n\\使用 中文 表述'

#### 完整语法：PromptTemplate

In [21]:
# 这是一个完整的函数调用，达到同样的效果
from langchain.prompts import PromptTemplate
PromptTemplate(
    input_variables=['language', 'topic'],
    output_parser=None,
    partial_variables={},
    template='告诉我一个关于{topic}的笑话, 一定要特别好笑\n\使用 {language} 表述',
    template_format='f-string',
    validate_template=True
)

PromptTemplate(input_variables=['language', 'topic'], template='告诉我一个关于{topic}的笑话, 一定要特别好笑\n\\使用 {language} 表述', validate_template=True)

#### 结合 Chain 使用

In [10]:
# 也可以直接在chain中给参数
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI

model = ChatOpenAI()
chain = LLMChain(llm=model, prompt=prompt,output_parser=parser)

chain.invoke({"topic": "足球", "language": "中文"})

{'topic': '足球',
 'language': '中文',
 'text': '为什么足球比赛中的球员总是穿着裤子？\n因为他们不想踢到“球”露出来！哈哈哈哈哈哈哈！'}

### 对话模板提示语

#### 基本语法

In [1]:
from langchain.schema import AIMessage, HumanMessage, SystemMessage
prompt = SystemMessage(content="You are a nice pirate")
new_prompt = (
    prompt + HumanMessage(content="hi") + AIMessage(content="what?") + "{input}"
)
new_prompt.format_messages(input="i said hi")

[SystemMessage(content='You are a nice pirate'),
 HumanMessage(content='hi'),
 AIMessage(content='what?'),
 HumanMessage(content='i said hi')]

#### 使用字典构建

In [13]:
from langchain_core.prompts import ChatPromptTemplate

template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI bot. Your name is {name}."),
    ("human", "Hello, how are you doing?"),
    ("ai", "I'm doing well, thanks!"),
    ("human", "{user_input}"),
])

template.format_messages(
    name="Bob",
    user_input="What is your name?"
)

[SystemMessage(content='You are a helpful AI bot. Your name is Bob.'),
 HumanMessage(content='Hello, how are you doing?'),
 AIMessage(content="I'm doing well, thanks!"),
 HumanMessage(content='What is your name?')]

In [12]:
from langchain.
resp.messages

[SystemMessage(content='You are a nice pirate'),
 HumanMessage(content='hi'),
 AIMessage(content='what?'),
 HumanMessage(content='i said hi')]

### 组装能力

#### 在提示语中填充例子

#### 在提示语中填充小样本

#### 局部修改提示语模板

In [80]:
# 通过局部修改实现提示语管理
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(template="{foo}{bar}", input_variables=["foo", "bar"])
partial_prompt = prompt.partial(foo="foo")
print(partial_prompt.format(bar="baz"))

foobaz


In [81]:
# 或者这样做
prompt = PromptTemplate(
    template="{foo}{bar}", input_variables=["bar"], partial_variables={"foo": "foo"}
)
print(prompt.format(bar="baz"))

foobaz


In [82]:
# 使用函数
from datetime import datetime

def _get_datetime():
    now = datetime.now()
    return now.strftime("%m/%d/%Y, %H:%M:%S")

prompt = PromptTemplate(
    template="Tell me a {adjective} joke about the day {date}",
    input_variables=["adjective", "date"],
)
partial_prompt = prompt.partial(date=_get_datetime)
print(partial_prompt.format(adjective="funny"))


Tell me a funny joke about the day 01/23/2024, 16:27:57


In [83]:
# 换个方式使用函数
prompt = PromptTemplate(
    template="Tell me a {adjective} joke about the day {date}",
    input_variables=["adjective"],
    partial_variables={"date": _get_datetime},
)
print(prompt.format(adjective="funny"))

Tell me a funny joke about the day 01/23/2024, 16:28:51


#### 提示语pipeline

In [69]:
from langchain.prompts.pipeline import PipelinePromptTemplate
from langchain.prompts.prompt import PromptTemplate

In [70]:
full_template = """{introduction}

{example}

{start}"""
full_prompt = PromptTemplate.from_template(full_template)

In [71]:
introduction_template = """You are impersonating {person}."""
introduction_prompt = PromptTemplate.from_template(introduction_template)

In [72]:
example_template = """Here's an example of an interaction:

Q: {example_q}
A: {example_a}"""
example_prompt = PromptTemplate.from_template(example_template)

In [73]:
start_template = """Now, do this for real!

Q: {input}
A:"""
start_prompt = PromptTemplate.from_template(start_template)

In [75]:
input_prompts = [
    ("introduction", introduction_prompt),
    ("example", example_prompt),
    ("start", start_prompt),
]
pipeline_prompt = PipelinePromptTemplate(
    final_prompt=full_prompt, pipeline_prompts=input_prompts
)

In [76]:
pipeline_prompt.input_variables

['person', 'example_a', 'example_q', 'input']

In [77]:
print(
    pipeline_prompt.format(
        person="Elon Musk",
        example_q="What's your favorite car?",
        example_a="Tesla",
        input="What's your favorite social media site?",
    )
)

You are impersonating Elon Musk.

Here's an example of an interaction:

Q: What's your favorite car?
A: Tesla

Now, do this for real!

Q: What's your favorite social media site?
A:


## 对话模型

### LCEL

对话模型实现了Runnable接口，并自动实现以下接口：

- invoke
- ainvoke
- stream
- astream
- batch
- abatch
- astream_log

In [86]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

chat = ChatOpenAI()
messages = [
    SystemMessage(content="You're a helpful assistant"),
    HumanMessage(content="模型为什么要做正则化?"),
]
chat.invoke(messages)

AIMessage(content='模型在训练数据上表现良好，但在新的未见过的数据上可能会出现过拟合（overfitting）的情况。正则化是一种用来解决过拟合问题的技术。它通过在损失函数中引入一个正则化项，限制模型的复杂度，防止模型过度拟合训练数据。\n\n正则化的目的是平衡模型的拟合能力和泛化能力。如果模型过于复杂，它可能会过度拟合训练数据，导致在新数据上的表现较差。正则化通过对模型参数的惩罚，鼓励模型选择更简单的参数组合，从而降低模型的复杂度。\n\n常见的正则化方法包括L1正则化和L2正则化。L1正则化通过在损失函数中添加模型参数的绝对值之和，促使模型参数稀疏化，即让一些参数变为0，从而实现特征选择的效果。L2正则化通过在损失函数中添加模型参数的平方和，降低参数的绝对值，使模型更加平滑。\n\n正则化可以帮助减少模型的方差，提高模型的泛化能力，从而在新的未见过的数据上表现更好。它是训练模型时常用的一种技术，可以提高模型的稳定性和可靠性。')

In [88]:
for chunk in chat.stream(messages):
    print(chunk.content, end="", flush=True)

模型正则化是为了减少过拟合（Overfitting）的发生。在训练模型时，如果模型过于复杂，容易出现过拟合的情况，即在训练集上表现很好，但在未知数据上表现较差。过拟合的原因是因为模型过度拟合了训练数据的噪声和细节，并且没有很好地学习到数据的普遍规律。

正则化通过在模型的损失函数中引入正则项，对模型的复杂度进行惩罚，从而降低模型的复杂度。正则化的目的是通过控制模型参数的大小，使模型更加简单，能够更好地泛化到未知数据上。常见的正则化方法有L1正则化和L2正则化。

L1正则化通过在损失函数中添加模型参数的L1范数（绝对值之和）作为正则项，可以使得模型的部分参数变为0，从而实现特征选择的效果，减少模型的复杂度。

L2正则化通过在损失函数中添加模型参数的L2范数（平方和的平方根）作为正则项，可以使得模型参数的值较小，从而降低模型的复杂度。

正则化可以在一定程度上防止过拟合，提高模型的泛化能力，使模型在未知数据上表现更好。

In [89]:
chat.batch([messages])

[AIMessage(content='模型正则化是为了解决过拟合问题。过拟合是指模型在训练数据上表现良好，但在新的未见过的数据上表现较差的现象。正则化通过在模型的损失函数中添加一个正则化项，惩罚模型的复杂度，从而限制模型的学习能力，减少模型对训练数据的过度拟合。\n\n正则化有助于提高模型的泛化能力，使其在新数据上的表现更好。常见的正则化方法包括L1正则化（Lasso）和L2正则化（Ridge），它们分别通过对模型的权重进行惩罚，降低模型的复杂度。正则化还可以用于特征选择，通过对特征的权重进行惩罚，减少对不相关特征的依赖。\n\n总之，模型正则化是为了防止过拟合，提高模型的泛化能力，从而使模型在未见过的数据上表现更好。')]

### 使用内存缓存

In [90]:
from langchain.globals import set_llm_cache
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

In [100]:
%%time
from langchain.cache import InMemoryCache

set_llm_cache(InMemoryCache())

# The first time, it is not yet in cache, so it should take longer
llm.invoke("Tell me a joke")

CPU times: user 11.2 ms, sys: 2.09 ms, total: 13.3 ms
Wall time: 1.98 s


AIMessage(content="Sure, here's a classic one for you:\n\nWhy don't scientists trust atoms?\n\nBecause they make up everything!")

下面的相同调用不会重复访问大模型：

In [102]:
%%time
# The second time it is, so it goes faster
llm.invoke("Tell me a joke")

CPU times: user 1.24 ms, sys: 101 µs, total: 1.34 ms
Wall time: 2.09 ms


AIMessage(content="Sure, here's a classic one for you:\n\nWhy don't scientists trust atoms?\n\nBecause they make up everything!")

### 使用SQLite缓存

In [103]:
!rm .langchain.db

rm: .langchain.db: No such file or directory


In [104]:
# We can do the same thing with a SQLite cache
from langchain.cache import SQLiteCache

set_llm_cache(SQLiteCache(database_path=".langchain.db"))

In [107]:
%%time
# The first time, it is not yet in cache, so it should take longer
llm.predict("Tell me a joke")

CPU times: user 3.23 ms, sys: 1.27 ms, total: 4.49 ms
Wall time: 3.59 ms


"Sure, here's a classic one for you:\n\nWhy don't scientists trust atoms?\n\nBecause they make up everything!"

### Token跟踪

In [110]:
from langchain.callbacks import get_openai_callback
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
with get_openai_callback() as cb:
    result = llm.invoke("Tell me a new joke")
    print(cb)

llm4 = ChatOpenAI(model_name="gpt-4")
with get_openai_callback() as cb:
    result = llm4.invoke("Tell me a new apple joke")
    print(cb)

Tokens Used: 35
	Prompt Tokens: 12
	Completion Tokens: 23
Successful Requests: 1
Total Cost (USD): $6.4e-05
Tokens Used: 32
	Prompt Tokens: 13
	Completion Tokens: 19
Successful Requests: 1
Total Cost (USD): $0.00153


## LLMs

### OpenAI封装

In [7]:
# 最简单的代码
from langchain_openai import ChatOpenAI

llm = ChatOpenAI() # 默认是gpt-3.5-turbo
response = llm.invoke("你是谁")
print(response.content)

我是一个AI助手，被称为OpenAI Assistant。我被设计用来回答各种问题和提供帮助。有什么我可以帮助你的吗？


### 通义千问封装

In [10]:
!poetry add dashscope

Using version ^1.14.0 for dashscope

Updating dependencies
Resolving dependencies... (0.8s)

Package operations: 1 install, 0 updates, 0 removals

  • Installing dashscope (1.14.0): Pending...
  • Installing dashscope (1.14.0): Downloading... 0%
  • Installing dashscope (1.14.0): Downloading... 80%
  • Installing dashscope (1.14.0): Downloading... 100%
  • Installing dashscope (1.14.0): Installing...
  • Installing dashscope (1.14.0)

Writing lock file


In [12]:
# 其它模型分装在 langchain_community 底包中
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.llms import Tongyi

ty_llm = Tongyi()
messages = [
    HumanMessage(content="你是谁") 
]
ty_llm.invoke(messages)

'我是阿里云开发的一款超大规模语言模型，我叫通义千问。'

### 自定义LLM

In [111]:
from typing import Any, List, Mapping, Optional
from langchain_core.callbacks.manager import CallbackManagerForLLMRun
from langchain_core.language_models.llms import LLM

In [118]:
# 实现一个定制的LLM接入
class CustomLLM(LLM):
    n: int

    @property
    def _llm_type(self) -> str:
        return "custom"

    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:
        if stop is not None:
            raise ValueError("stop kwargs are not permitted.")
        return prompt[: self.n]

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        """Get the identifying parameters."""
        return {"n": self.n}

In [113]:
llm = CustomLLM(n=10)

In [114]:
llm.invoke("This is a foobar thing")

'This is a '

In [117]:
print(llm)

CustomLLM
Params: {'n': 10}


## 从文件加载提示语模板

### yaml格式

In [ ]:
 _type: prompt
input_variables:
    ["adjective", "content"]
template: 
    Tell me a {adjective} joke about {content}.

### json格式

In [16]:
{
    "_type": "prompt",
    "input_variables": ["adjective", "content"],
    "template": "Tell me a {adjective} joke about {content}."
}

{'_type': 'prompt',
 'input_variables': ['adjective', 'content'],
 'template': 'Tell me a {adjective} joke about {content}.'}

### json + txt

首先，将模板主要内容写入**final_step.txt**：

然后，在**task.json**文件中指定**template_path**嵌入路径：

In [43]:
{
    "_type": "prompt",
    "input_variables": [
      "ai_name",
      "ai_role",
      "task_description",
      "short_term_memory"
    ],
    "template_path": "final_step.txt"
}

{'_type': 'prompt',
 'input_variables': ['ai_name',
  'ai_role',
  'task_description',
  'short_term_memory'],
 'template_path': 'final_step.txt'}

### 加载提示语模板文件

In [17]:
from langchain.prompts import load_prompt

prompt = load_prompt("simple_prompt.json")
print(prompt.format(adjective="funny", content="Xiao Ming"))

Tell me a funny joke about Xiao Ming.


## OutputParser

自动把 LLM 输出的字符串按指定格式加载。

LangChain 内置的 OutputParser 包括:

- StrOutputParser
- OpenAIFunctions
- ListParser
- DatetimeParser
- EnumParser
- PydanticParser
- XMLParser
等等

### JSON parser

In [135]:
from typing import List
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

model = ChatOpenAI(temperature=0)

# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

In [138]:
# And a query intented to prompt a language model to populate the data structure.
joke_query = "Tell me a joke."

# Set up a parser + inject instructions into the prompt template.
parser = JsonOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser

print(prompt)
chain.invoke({"query": joke_query})

input_variables=['query'] partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"setup": {"title": "Setup", "description": "question to set up a joke", "type": "string"}, "punchline": {"title": "Punchline", "description": "answer to resolve the joke", "type": "string"}}, "required": ["setup", "punchline"]}\n```'} template='Answer the user query.\n{format_instructions}\n{query}\n'


{'setup': "Why don't scientists trust atoms?",
 'punchline': 'Because they make up everything!'}

### OpenAI Functions

从 pydantic 转换 openai 函数名和参数：

In [140]:
from langchain_community.utils.openai_functions import (
    convert_pydantic_to_openai_function,
)
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_openai import ChatOpenAI

class Joke(BaseModel):
    """Joke to tell user."""

    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

openai_functions = [convert_pydantic_to_openai_function(Joke)]

In [141]:
model = ChatOpenAI(temperature=0)
prompt = ChatPromptTemplate.from_messages(
    [("system", "You are helpful assistant"), ("user", "{input}")]
)

**JsonOutputFunctionsParser**

In [144]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
parser = JsonOutputFunctionsParser()

# 绑定openai函数，并使用json解析出参数
chain = prompt | model.bind(functions=openai_functions) | parser

In [143]:
chain.invoke({"input": "tell me a joke"})

{'setup': "Why don't scientists trust atoms?",
 'punchline': 'Because they make up everything!'}

**JsonKeyOutputFunctionsParser**

In [147]:
from typing import List
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser

class Jokes(BaseModel):
    """Jokes to tell user."""

    joke: List[Joke]
    funniness_level: int

In [148]:
parser = JsonKeyOutputFunctionsParser(key_name="joke")

In [149]:
openai_functions = [convert_pydantic_to_openai_function(Jokes)]
chain = prompt | model.bind(functions=openai_functions) | parser

In [150]:
chain.invoke({"input": "tell me two jokes"})

[{'setup': "Why don't scientists trust atoms?",
  'punchline': 'Because they make up everything!'},
 {'setup': 'Why did the scarecrow win an award?',
  'punchline': 'Because he was outstanding in his field!'}]

In [151]:
for s in chain.stream({"input": "tell me two jokes"}):
    print(s)

[]
[{}]
[{'setup': ''}]
[{'setup': 'Why'}]
[{'setup': 'Why don'}]
[{'setup': "Why don't"}]
[{'setup': "Why don't scientists"}]
[{'setup': "Why don't scientists trust"}]
[{'setup': "Why don't scientists trust atoms"}]
[{'setup': "Why don't scientists trust atoms?"}]
[{'setup': "Why don't scientists trust atoms?", 'punchline': ''}]
[{'setup': "Why don't scientists trust atoms?", 'punchline': 'Because'}]
[{'setup': "Why don't scientists trust atoms?", 'punchline': 'Because they'}]
[{'setup': "Why don't scientists trust atoms?", 'punchline': 'Because they make'}]
[{'setup': "Why don't scientists trust atoms?", 'punchline': 'Because they make up'}]
[{'setup': "Why don't scientists trust atoms?", 'punchline': 'Because they make up everything'}]
[{'setup': "Why don't scientists trust atoms?", 'punchline': 'Because they make up everything!'}]
[{'setup': "Why don't scientists trust atoms?", 'punchline': 'Because they make up everything!'}, {}]
[{'setup': "Why don't scientists trust atoms?", 'pu

### Enum parser

In [130]:
from langchain.output_parsers.enum import EnumOutputParser
from enum import Enum

class Colors(Enum):
    RED = "red"
    GREEN = "green"
    BLUE = "blue"

parser = EnumOutputParser(enum=Colors)

In [131]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

prompt = PromptTemplate.from_template(
    """What color eyes does this person have?

> Person: {person}

Instructions: {instructions}"""
).partial(instructions=parser.get_format_instructions())
chain = prompt | ChatOpenAI() | parser

In [134]:
print(prompt)
chain.invoke({"person": "Frank Sinatra"})

input_variables=['person'] partial_variables={'instructions': 'Select one of the following options: red, green, blue'} template='What color eyes does this person have?\n\n> Person: {person}\n\nInstructions: {instructions}'


<Colors.BLUE: 'blue'>

### Structured output parser

### YAML parser

### XML parser

### Datetime parser

In [122]:
from langchain.output_parsers import DatetimeOutputParser
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI

In [123]:
output_parser = DatetimeOutputParser()
template = """Answer the users question:

{question}

{format_instructions}"""
prompt = PromptTemplate.from_template(
    template,
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)

In [126]:
prompt

PromptTemplate(input_variables=['question'], partial_variables={'format_instructions': "Write a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.\n\nExamples: 0126-01-12T17:02:00.512595Z, 0719-05-06T14:30:04.335045Z, 1391-07-06T00:25:48.331596Z\n\nReturn ONLY this string, no other words!"}, template='Answer the users question:\n\n{question}\n\n{format_instructions}')

In [129]:
chain = prompt | OpenAI() | output_parser
output = chain.invoke({"question": "when was bitcoin founded?"})
print(output)

2009-01-03 18:15:05


### Pydantic parser

### CSV parser

In [119]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

output_parser = CommaSeparatedListOutputParser()

format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template="List five {subject}.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions},
)

model = ChatOpenAI(temperature=0)

chain = prompt | model | output_parser

In [120]:
chain.invoke({"subject": "ice cream flavors"})

['Vanilla',
 'Chocolate',
 'Strawberry',
 'Mint Chocolate Chip',
 'Cookies and Cream']

In [121]:
for s in chain.stream({"subject": "ice cream flavors"}):
    print(s)

['Vanilla']
['Chocolate']
['Strawberry']
['Mint Chocolate Chip']
['Cookies and Cream']


### Pandas DataFrame Parser

### Output-fixing parser

### Retry parser

## 链式调用

可以直接使用 **LCEL** 语法构建自己的链，也可以使用现成的。
使用前最好直接查看 **langchain** 源代码。
- 使用 OpenAI function calling
- 创建数据库查询
- 检索文档
- ...

In [ ]:
from langchain_community.chat_models import ChatOpenAI
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain import hub

retrieval_qa_chat_prompt = hub.pull("langchain-ai/retrieval-qa-chat")
llm = ChatOpenAI()
retriever = ...
combine_docs_chain = create_stuff_documents_chain(
    llm, retrieval_qa_chat_prompt
)
retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)

chain.invoke({"input": "..."})

## 记忆封装

### Tools

## RAG的例子

In [21]:
!poetry add "langchain[docarray]"

Using version ^0.1.3 for langchain

Updating dependencies
Resolving dependencies... (14.7s)Resolving dependencies... (9.5s)Resolving dependencies... (9.6s)

No dependencies to install or update


In [ ]:
# Requires:
# pip install langchain docarray tiktoken

from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings

vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()
output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)
chain = setup_and_retrieval | prompt | model | output_parser

chain.invoke("where did harrison work?")

# 综合实践

## 搜索引擎

In [152]:
!poetry add duckduckgo-search

Using version ^4.3 for duckduckgo-search

Updating dependencies
Resolving dependencies... (20.5s)Resolving dependencies... (11.7s)Resolving dependencies... (14.6s)Resolving dependencies... (14.9s)Resolving dependencies... (15.0s)Resolving dependencies... (17.0s)Resolving dependencies... (17.1s)Resolving dependencies... (18.3s)Resolving dependencies... (20.6s)

Package operations: 3 installs, 0 updates, 0 removals

  • Installing curl-cffi (0.6.0b7): Pending...
  • Installing docstring-inheritance (2.1.2): Pending...
  • Installing docstring-inheritance (2.1.2): Downloading... 0%
  • Installing docstring-inheritance (2.1.2): Downloading... 100%
  • Installing docstring-inheritance (2.1.2): Installing...
  • Installing docstring-inheritance (2.1.2)
  • Installing docstring-inheritance (2.1.2)
  • Installing curl-cffi (0.6.0b7): Downloading... 0%
  • Installing docstring-inheritance (2.1.2)
  • Installing docstring-inheritance (2.1.2)
  • Installing curl-cffi (0.6.0b7): Downloading... 10%

In [161]:
from langchain.tools import DuckDuckGoSearchRun
import nest_asyncio
nest_asyncio.apply()

In [162]:
search = DuckDuckGoSearchRun()

In [ ]:
search.run("Obama's first name?")

## 通过文本向量路由Prompt

In [107]:
from langchain.utils.math import cosine_similarity
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

physics_template = """你是一个物理学教授，负责给数学爱好者解答疑惑。\
你正在为小学生回答问题，注意使用小学生水平能听懂的词汇，避免过于专业晦涩的术语。 \
当你不知道答案时就回答不知道。

Here is a question:
{query}"""

math_template = """你是一个数学家，负责给小学生解答疑惑。
注意使用小学生水平能听懂的词汇，避免过于专业晦涩的术语。 \
回答时，请举一些生活中的例子。
当你不知道答案时就回答不知道。

Here is a question:
{query}"""

embeddings = OpenAIEmbeddings()
prompt_templates = [physics_template, math_template]
prompt_embeddings = embeddings.embed_documents(prompt_templates)


def prompt_router(input):
    query_embedding = embeddings.embed_query(input["query"])
    similarity = cosine_similarity([query_embedding], prompt_embeddings)[0]
    most_similar = prompt_templates[similarity.argmax()]
    print("Using MATH" if most_similar == math_template else "Using PHYSICS")
    return PromptTemplate.from_template(most_similar)

chain = (
    {"query": RunnablePassthrough()}
    | RunnableLambda(prompt_router)
    | ChatOpenAI()
    | StrOutputParser()
)

common_train = ChatOpenAI() | StrOutputParser()

In [105]:
print(common_train.invoke("黑洞是什么？"))

黑洞是一种极度密集的天体，它具有非常强大的引力场，以至于连光都无法逃离它的吸引。黑洞的形成是由于一个恒星在死亡时，其质量过大，无法通过核聚变维持稳定，导致恒星坍缩成一个极为紧凑的物体。黑洞的中心部分称为奇点，奇点的密度和引力非常之大，超过了任何已知物质的极限。

黑洞的存在可以通过它们产生的引力效应来间接观测，例如吸收附近的物质、扭曲周围空间和发射强烈的辐射。虽然我们无法直接观测到黑洞，但科学家们通过观测它们对周围物体的影响，以及通过天文观测和数学模型来研究黑洞的性质和行为。

黑洞在宇宙中广泛存在，它们可能是恒星坍缩形成的中等质量黑洞，也可能是超大质量黑洞，如位于银河系中心的超大质量黑洞。黑洞对宇宙的演化和结构具有重要影响，它们是天体物理学和相对论研究的重要对象。


In [106]:
print(chain.invoke("黑洞是什么？"))

Using PHYSICS
小朋友，黑洞是宇宙中一种非常特殊的东西。它是一种非常强大的引力场，就像一个很大的吸力。当一颗非常大的恒星（就是我们看到的星星）燃烧完燃料后，它会塌缩成一个非常小又非常密集的东西，就是黑洞。黑洞的引力非常强大，甚至连光也无法逃脱它的吸引力。所以我们看不到黑洞，它是非常神秘的。关于黑洞，科学家们还在研究中，有很多有趣的发现等待我们去探索。


In [108]:
print(common_train.invoke("路径积分是什么？"))

路径积分是一个物理学概念，用来描述在一个力场中沿着一条曲线路径上的力的积累效果。简单来说，路径积分是将一个向量场沿着一条曲线进行积分，得到沿着该曲线的总体积效应。

在物理学中，路径积分可以用来计算沿着一个曲线路径上的力的总效果，比如沿着一条曲线上的力的总功或者总位移。路径积分的计算方式是将力场在曲线上的每个点上的力与微小位移相乘，然后将所有微小的力与位移的乘积相加，得到曲线上的总效果。

路径积分在许多领域中都有重要的应用，比如在力学中用于计算物体在曲线路径上的总功、在电磁学中用于计算电场或磁场沿着曲线的总位移等。路径积分的计算可以通过数学上的积分运算来实现，根据具体情况可以采用不同的积分方法，比如定积分或线积分等。


In [109]:
print(chain.invoke("路径积分是什么？"))

Using MATH
路径积分是一种数学工具，它在物理学中常常被用来描述粒子或光在空间中的运动。你可以把路径积分想象成一个粒子或光在不同路径上行走的概率，就像我们在城市里选择不同的路线去目的地一样。

想象一下你要从学校回家，有很多条路可以选择。每条路都有不同的长度、不同的交通状况和不同的风景。路径积分就是用来计算你选择每条路的概率，也就是说，你走每条路的可能性有多大。

在物理学中，粒子或光在空间中运动的时候，也有很多可能的路径可以选择。路径积分可以帮助我们计算出每条路径的概率，从而更好地理解粒子或光的行为。

但是，具体如何计算路径积分，需要更深入的数学知识和物理背景。这里只是简单介绍了路径积分的概念，如果你对它感兴趣，可以在以后的学习中深入了解。


## 执行python代码

In [110]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import (
    ChatPromptTemplate,
)
from langchain_experimental.utilities import PythonREPL
from langchain_openai import ChatOpenAI

In [123]:
template = """Write some python code to solve the user's problem. 

Return only python code in Markdown format and Chinese, e.g.:

```python
....
```"""
prompt = ChatPromptTemplate.from_messages([("system", template), ("human", "{input}")])

model = ChatOpenAI()

In [124]:
def _sanitize_output(text: str):
    print(text)
    _, after = text.split("```python")
    return after.split("```")[0]

In [125]:
chain = prompt | model | StrOutputParser() | _sanitize_output | PythonREPL().run

In [126]:
chain.invoke({"input": "一个笼子里有兔子和鸡若干，数一数有5个头，12只脚，请问有多少只兔子多少只鸡？"})

我们可以使用穷举法来解决这个问题。

假设有 x 只兔子，y 只鸡。根据题意，可以得到以下两个方程：

x + y = 5   # 头的数量
4x + 2y = 12  # 脚的数量

我们可以通过求解这个方程组来得到兔子和鸡的数量。让我们来编写代码实现这个算法。

```python
def solve():
    for x in range(6):  # 兔子的数量最多为5只
        y = 5 - x  # 根据第一个方程计算鸡的数量
        if 4*x + 2*y == 12:  # 检查第二个方程是否满足
            return x, y  # 返回兔子和鸡的数量

rabbit, chicken = solve()
print(f"兔子的数量为：{rabbit} 只，鸡的数量为：{chicken} 只")
```

运行这段代码，我们可以得到输出：

```
兔子的数量为：1 只，鸡的数量为：4 只
```

所以，笼子里有1只兔子和4只鸡。


'兔子的数量为：1 只，鸡的数量为：4 只\n'

## 查询数据库

In [132]:
from langchain_core.prompts import ChatPromptTemplate

template = """Based on the table schema below, write a SQL query that would answer the user's question:
{schema}

Question: {question}
SQL Query:"""
prompt = ChatPromptTemplate.from_template(template)

In [133]:
from langchain_community.utilities import SQLDatabase

In [134]:
db = SQLDatabase.from_uri("sqlite:///./Chinook.sqlite")

In [135]:
def get_schema(_):
    return db.get_table_info()

In [136]:
def run_query(query):
    return db.run(query)

In [137]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

model = ChatOpenAI()

sql_response = (
    RunnablePassthrough.assign(schema=get_schema)
    | prompt
    | model.bind(stop=["\nSQLResult:"])
    | StrOutputParser()
)

In [138]:
## 直接生成查询语句
sql_response.invoke({"question": "How many employees are there?"})

'SELECT COUNT(*) FROM Employee'

In [149]:
template = """Based on the table schema below, question, sql query, and sql response, write a natural language response:
{schema}

Question: {question}
SQL Query: {query}
SQL Response: {response}

请用中文回答。
"""
prompt_response = ChatPromptTemplate.from_template(template)

In [150]:
# 注意要分两阶段执行assign：先生成SQL，才能执行SQL
full_chain = (
    RunnablePassthrough.assign(
        schema=get_schema,
        query=sql_response
    ).assign(
        response=lambda x: db.run(x["query"]),
    )
    | prompt_response
    | model
)

In [151]:
full_chain.invoke({"question": "员工人数是多少?"})

AIMessage(content='员工人数是8人。')

# 集成langfuse

In [2]:
from langchain_openai import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langfuse.callback import CallbackHandler
import uuid

In [4]:
handler = CallbackHandler(trace_name="learning-langchain", user_id="homeway", session_id=str(uuid.uuid4()))

In [6]:
llm = ChatOpenAI(model = "gpt-3.5-turbo", streaming = False, temperature = 0.5)
parser = StrOutputParser()
prompt = ChatPromptTemplate.from_template("hi")
train = (prompt | llm | parser)
train.invoke({}, config = {"callbacks": [handler]})

'Hello! How can I assist you today?'

In [8]:
# 多轮对话
from langchain.schema import (
    AIMessage, #等价于OpenAI接口中的 assistant role
    HumanMessage, #等价于OpenAI接口中的 user role
    SystemMessage #等价于OpenAI接口中的 system role
)

messages = [
    SystemMessage(content="你是AGIClass的课程助理。"), 
    HumanMessage(content="我是学员，我叫薛宏伟。"), 
    AIMessage(content="欢迎！"),
    HumanMessage(content="我是谁") 
]
llm.invoke(messages) 

AIMessage(content='你是薛宏伟。')

In [15]:
# 对话提示语模板
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.chat import SystemMessagePromptTemplate, HumanMessagePromptTemplate

template = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template("你是{product}的客服助手。你的名字叫{name}"),
        HumanMessagePromptTemplate.from_template("{query}"),
    ]
)

llm = ChatOpenAI()
prompt = template.format_messages(
        product="广州鸿蒙",
        name="蒙蒙",
        query="你是谁"
    )

llm.invoke(prompt)

AIMessage(content='我是广州鸿蒙的客服助手，名字叫蒙蒙。有什么可以帮到您的吗？')

# 集成langserve

## 与fastapi一起使用

In [ ]:
#!/usr/bin/env python
from fastapi import FastAPI
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langserve import add_routes

app = FastAPI(
    title="LangChain Server",
    version="1.0",
    description="A simple api server using Langchain's Runnable interfaces",
)

add_routes(
    app,
    ChatOpenAI(),
    path="/openai",
)

model = ChatOpenAI()
prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
add_routes(
    app,
    prompt | model,
    path="/joke",
)

if __name__ == "__main__":
    import uvicorn

    uvicorn.run(app, host="localhost", port=8000)

## 与langfuse一起使用

In [ ]:
handler = CallbackHandler(trace_name="chat_once", user_id="wencheng")
prompt = ChatPromptTemplate.from_template(
    """{question}""")
llm = ChatOpenAI(model = "gpt-3.5-turbo-16k", streaming = True, temperature = 0)
chain = (prompt | llm | parser).with_config({"callbacks": [handler]})

add_routes(app, chain, path = "/langserve/chat_once")

## python Client

In [67]:
!poetry add httpx_sse

Using version ^0.4.0 for httpx-sse

Updating dependencies
Resolving dependencies... (16.3s)Resolving dependencies... (0.1s)Resolving dependencies... (7.9s)Resolving dependencies... (9.3s)

Package operations: 1 install, 0 updates, 0 removals

  • Installing httpx-sse (0.4.0): Pending...
  • Installing httpx-sse (0.4.0): Installing...
  • Installing httpx-sse (0.4.0)

Writing lock file


In [64]:
!poetry add langserve

The following packages are already present in the pyproject.toml and will be skipped:

  • langserve

If you want to update it to the latest compatible version, you can use `poetry update package`.
If you prefer to upgrade it to the latest available version, you can use `poetry add package@latest`.

Nothing to add.


In [70]:
from langserve import RemoteRunnable
chat_once = RemoteRunnable("http://localhost:8000/langserve/chat_once")

东莞是中国广东省下辖的一个地级市，位于珠江三角洲南部，东临深圳，西接广州，北邻惠州，南濒珠江口。作为中国改革开放的重要窗口和制造业基地，东莞是中国最重要的制造业城市之一。

东莞是中国最早的经济特区之一，也是中国最大的制造业城市之一。它以制造业为主导，涵盖了电子、电器、纺织、玩具、家具、鞋业等多个行业。许多国内外知名品牌都在东莞设有生产基地。东莞的制造业发展水平和产业链完善程度在全国具有较高的竞争力。

除了制造业，东莞也在不断发展其他产业，如现代服务业、高新技术产业和文化创意产业等。近年来，东莞还加大了对科技创新的投入，积极推动产业升级和转型发展。

东莞也是一个宜居的城市，拥有良好的基础设施和公共服务。城市规划合理，交通便利，医疗、教育、文化等公共服务设施完善。同时，东莞还注重生态环境保护，积极推动绿色发展，建设了许多公园和绿地，提供了良好的生活环境。

此外，东莞还有一些旅游景点值得一提。如虎门石龙山、广东现代国际展览中心、东莞松山湖科技产业园等。这些景点展示了东莞的自然风光和城市发展成果。

总的来说，东莞是一个以制造业为主导的现代化城市，拥有发达的经济和良好的生活环境。无论是商务出差还是旅游观光，东莞都是一个值得一去的地方。

In [71]:
chat_once.invoke({"question": "能帮我介绍一下东莞吗？"})

'东莞是中国广东省下辖的一个地级市，位于珠江三角洲南部，东临深圳，西接广州，北邻惠州，南濒珠江口。作为中国改革开放的重要窗口和制造业基地，东莞是中国最重要的制造业城市之一。\n\n东莞是中国最早的经济特区之一，也是中国最大的制造业城市之一。它以制造业为主导，涵盖了电子、电器、纺织、玩具、家具、鞋业等多个行业。许多国内外知名品牌都在东莞设有生产基地。东莞的制造业发展水平和产业链完善程度在全国具有较高的竞争力。\n\n除了制造业，东莞也在不断发展其他产业，如现代服务业、高新技术产业和文化创意产业等。近年来，东莞还加大了对科技创新的投入，积极推动产业升级和转型发展。\n\n东莞也是一个宜居的城市，拥有良好的基础设施和公共服务。城市规划合理，交通便利，医疗、教育、文化等公共服务设施完善。同时，东莞还注重生态环境保护，积极推动绿色发展，建设了许多公园和绿地，提供了良好的生活环境。\n\n此外，东莞还有一些旅游景点值得一提。如虎门石龙山、广东现代国际展览中心、东莞松山湖科技产业园等。这些景点展示了东莞的自然风光和城市发展成果。\n\n总的来说，东莞是一个以制造业为主导的现代化城市，拥有发达的经济和良好的生活环境。无论是商务出差还是旅游观光，东莞都是一个值得一去的地方。'

In [ ]:
for chunk in chat_once.stream({"question": "能帮我介绍一下东莞吗？"}):
    print(chunk, end="", flush=True)

# javascript client

In [ ]:
!yarn add langchain

## 调用invoke

In [ ]:
import { RemoteRunnable } from "langchain/runnables/remote";

const remoteChain = new RemoteRunnable({
  url: "https://your_hostname.com/path",
});

const result = await remoteChain.invoke({
  param1: "param1",
  param2: "param2",
});

console.log(result);

## 调用stream

In [ ]:
const stream = await remoteChain.stream({
  param1: "param1",
  param2: "param2",
});

for await (const chunk of stream) {
  console.log(chunk);
}

## 使用config

In [ ]:
import { RemoteRunnable } from "langchain/runnables/remote";

const remoteChain = new RemoteRunnable({
  url: "https://your_hostname.com/path",
  options: {
    timeout: 10000,
    headers: {
      Authorization: "Bearer YOUR_TOKEN",
    },
  },
});

const result = await remoteChain.invoke({
  param1: "param1",
  param2: "param2",
});

console.log(result);